In [1]:
from huggingface_hub import hf_hub_download
from utils import load_model, load_model_skyclip, load_model_skyclip_2, load_xview_dataset, get_text_features
from evaluation import image_level_test
import os
import torch

/home/quoc/anaconda3/envs/geochat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 0. Load dataset

In [2]:
df, id_label_dict, superclass_mapped, dictionary=load_xview_dataset()

#### 1. RemoteCLIP
**a. Download model** (For the first time!)

Create a folder for the project and folder for downloading models
```
!mkdir RS-Foundation-Model
!mkdir foundation_models
!mkdir foundation_models/remote_sensing
!mkdir foundation_models/remote_sensing/RemoteCLIP
!mkdir foundation_models/remote_sensing/GeoRSCLIP
!mkdir foundation_models/remote_sensing/SkyCLIP
```

In [2]:
checkpoint_path = hf_hub_download("chendelong/RemoteCLIP", "RemoteCLIP-ViT-L-14.pt", cache_dir='checkpoints')

In [ ]:
!mv checkpoints/ ../foundation_models/remote_sensing/RemoteCLIP/

checkpoints/models--chendelong--RemoteCLIP/snapshots/bf1d8a3ccf2ddbf7c875705e46373bfe542bce38/RemoteCLIP-ViT-L-14.pt


**b. Load model**

In [4]:
folder = "../foundation_models/remote_sensing/RemoteCLIP/"
filename = "checkpoints/models--chendelong--RemoteCLIP/snapshots/bf1d8a3ccf2ddbf7c875705e46373bfe542bce38/RemoteCLIP-ViT-L-14.pt"
checkpoint_path = os.path.join(folder, filename)
model_name = 'ViT-L-14'
device = 'cuda' if torch.cuda.is_available else 'cpu'
checkpoint_path = checkpoint_path
model, preprocess, tokenizer=load_model(model_name, device, checkpoint_path)

/home/quoc/anaconda3/envs/geochat/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
text_features = get_text_features(model, tokenizer, dictionary)

/home/quoc/anaconda3/envs/geochat/lib/python3.10/site-packages/torch/nn/modules/activation.py:1160: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(


In [5]:
olabel = 1
image_level_test(model, preprocess, text_features, olabel, df)

100%|██████████| 57/57 [00:16<00:00,  3.37it/s]

top_1_accuracy=15.789473684210526, top_5_accuracy=64.91228070175438


(15.789473684210526, 64.91228070175438)

#### 2. GeoRSCLIP
**a. Download model** (For the first time!)

In [ ]:
!git clone https://huggingface.co/Zilun/GeoRSCLIP

In [ ]:
%rm -r GeoRSCLIP/ckpt/RS5M_ViT-B-32*.pt
%rm -r GeoRSCLIP/ckpt/RS5M_ViT-L-14-336.pt
%mkdir models/GeoRSCLIP
%mv /kaggle/working/RS-Foundation-Model/GeoRSCLIP/ckpt/* models/GeoRSCLIP/
%rm -r GeoRSCLIP

**b. Load model**

In [5]:
folder = "../foundation_models/remote_sensing/GeoRSCLIP/"
filename = "RS5M_ViT-L-14.pt"
checkpoint_path = os.path.join(folder, filename)
model_name = 'ViT-L-14'
device = 'cuda' if torch.cuda.is_available else 'cpu'
model, preprocess, tokenizer=load_model(model_name, device, checkpoint_path)

In [7]:
text_features = get_text_features(model, tokenizer, dictionary)

/home/quoc/anaconda3/envs/geochat/lib/python3.10/site-packages/torch/nn/modules/activation.py:1160: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(


In [8]:
olabel = 1
image_level_test(model, preprocess, text_features, olabel, df)

  0%|          | 0/57 [00:00<?, ?it/s]

100%|██████████| 57/57 [00:04<00:00, 12.75it/s]

top_1_accuracy=57.89473684210527, top_5_accuracy=94.73684210526315


(57.89473684210527, 94.73684210526315)

#### 3. SkyCLIP
**a. Download model** (For the first time!)

In [ ]:
MODEL_NAME = "SkyCLIP_ViT_L14_top30pct_filtered_by_CLIP_laion_RS.zip"
!curl -O https://opendatasharing.s3.us-west-2.amazonaws.com/SkyScript/ckpt/{MODEL_NAME}
!mv {MODEL_NAME} foundation_models/remote_sensing/SkyCLIP/
!unzip foundation_models/remote_sensing/SkyCLIP/SkyCLIP_ViT_L14_top30pct_filtered_by_CLIP_laion_RS.zip -d models/SkyCLIP/
!rm foundation_models/remote_sensing/SkyCLIP/SkyCLIP_ViT_L14_top30pct_filtered_by_CLIP_laion_RS.zip

**b. Load model**

In [6]:
folder = "../foundation_models/remote_sensing/SkyCLIP/"
filename = "SkyCLIP_ViT_L14_top30pct_filtered_by_CLIP_laion_RS/epoch_20.pt"
checkpoint_path = os.path.join(folder, filename)
model_name = 'ViT-L-14'
device = 'cuda' if torch.cuda.is_available else 'cpu'
src_path_to_skyscript = "SkyScript"
model, preprocess, tokenizer=load_model_skyclip(model_name, device, checkpoint_path, src_path_to_skyscript)

In [10]:
text_features = get_text_features(model, tokenizer, dictionary)

In [11]:
olabel = 1
image_level_test(model, preprocess, text_features, olabel, df)

  0%|          | 0/57 [00:00<?, ?it/s]

100%|██████████| 57/57 [00:14<00:00,  3.83it/s]

top_1_accuracy=22.807017543859647, top_5_accuracy=87.71929824561403


(22.807017543859647, 87.71929824561403)